In [ ]:
!pip install telethon
!pip install nest_asyncio
!pip install openpyxl
!pip install telethon nest_asyncio openpyxl
!pip install telethon nest_asyncio openpyxl nltk transformers sentence-transformers
!pip install textblob
!python -m textblob.download_corpora

Posts and Comments Extraction

In [ ]:

import nest_asyncio
import pandas as pd
import asyncio
from telethon import TelegramClient
from telethon.errors import MsgIdInvalidError
from datetime import datetime, timezone

nest_asyncio.apply()
get them from telegram.
api_id = 28524658
api_hash = '5117388049357ea47ab79365e82750ae'
create session to save login informations.
client = TelegramClient('YD_Cryptocurrency', api_id, api_hash)
Here define the date scope of posts
start_date = datetime(2023, 1, 15, 1, 9, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 2, 17, 12, 8, 7, tzinfo=timezone.utc)
Function to fetch and count post emoji reactions

def count_post_reactions(reactions):
    counts = {
        'Heart': 0, 'Thumbs Up': 0, 'Thumbs Down': 0,
        'Clapping Hands': 0, 'Exploding Head': 0, 'Broken Heart': 0
    }

    emoji_map = {
        '❤': 'Heart', '👍': 'Thumbs Up', '👎': 'Thumbs Down',
        '👏': 'Clapping Hands', '🤯': 'Exploding Head', '💔': 'Broken Heart'
    }

    if reactions:
        for reaction in reactions.results:
            emoji = getattr(reaction.reaction, 'emoticon', None) or getattr(reaction.reaction, 'emoji', None)
            if emoji in emoji_map:
                counts[emoji_map[emoji]] += reaction.count

    return counts

function to fetch and count comment emoji reactions.

def count_comment_reactions(reactions):
    counts = {
        'Heart': 0, 'Thumbs Up': 0
    }

    emoji_map = {
        '❤': 'Heart', '👍': 'Thumbs Up'
    }
    if reactions:
        for reaction in reactions.results:
            emoji = getattr(reaction.reaction, 'emoticon', None) or \
                    getattr(reaction.reaction, 'emoji', None)
            if emoji == '❤':
                counts['Heart'] += reaction.count
            # elif emoji == '🙏':
            #     counts['Pray'] += reaction.count
            elif emoji == '👍':
                counts['Thumbs Up'] += reaction.count
            # elif emoji == '🥰':
            #     counts['Face with Hearts'] += reaction.count

    return counts

this function used when the post has no comments.
Function to fetch all data from the Telegram channel
this main function used when work on colab or jupyter notebook to run the client and fetch data
async def fetch_comments_for_posts(posts, channel):
    tasks = [client.get_messages(channel, reply_to=post.id, limit=100) for post in posts]
    return await asyncio.gather(*tasks, return_exceptions=True)

async def fetch_data_with_comments(channel_username, batch_size=50, max_messages=400):
    channel = await client.get_entity(channel_username)
    posts = await fetch_data_from_channel(channel, batch_size=batch_size, max_messages=max_messages)

    print("Fetching comments for all posts...")
    comments_data = await fetch_comments_for_posts(posts, channel)

    structured_data = []
    for post, comments in zip(posts, comments_data):
        if post.text:
            post_content = post.text.strip('**')
            word_count = len(post_content.split())
            avg_words_per_sentence = word_count / max(1, post_content.count('.') + post_content.count('!') + post_content.count('?'))
            post_reactions = count_post_reactions(post.reactions)

            comments_details = []
            if isinstance(comments, list):
                for comment in comments:
                    if comment.text:
                        comment_reactions = count_comment_reactions(comment.reactions)
                        comments_details.append({
                            'Comment ID': comment.id,
                            'Comment Text': comment.text,
                            'Comment Time': comment.date.isoformat(),
                            **comment_reactions
                        })

            structured_data.append({
                'Post ID': post.id,
                'Post Content': post_content,
                'Post Time': post.date.isoformat(),
                'Views': post.views or 0,
                'Forwards': post.forwards or 0,
                **post_reactions,
                'Word Count': word_count,
                'Average Words per Sentence': avg_words_per_sentence,
                'Comments Details': comments_details
            })

    print(f"✅ Total structured posts fetched: {len(structured_data)}")
    return structured_data

def save_to_excel(data, filename='ناشيونال جيوغرافيگ.xlsx'):
    rows = []
    for post in data:
        if 'Comments Details' not in post or not post['Comments Details']:
            post['Comments Details'] = [{'Comment ID': None, 'Comment Text': None, 'Comment Time': None,
                                         'Heart': 0,
                                         'Thumbs Up': 0,}]

        for comment in post['Comments Details']:
            rows.append({
                'Post ID': post['Post ID'],
                'Post Content': post['Post Content'],
                'Post Time': post['Post Time'],
                'Views': post['Views'],
                'Forwards': post['Forwards'],
                **{key + ' (Post)': post.get(key, 0) for key in ['Heart', 'Thumbs Up', 'Thumbs Down', 'Clapping Hands', 'Exploding Head', 'Broken Heart']},
                'Word Count': post['Word Count'],
                'Average Words per Sentence': post['Average Words per Sentence'],
                'Comment ID': comment['Comment ID'],
                'Comment Text': comment['Comment Text'],
                'Comment Time': comment['Comment Time'],
                **{key + ' (Comment)': comment.get(key, 0) for key in ['Thumbs Up','Heart']}
            })

    df = pd.DataFrame(rows)
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"✅ Data successfully saved to {filename}")

async def main():
    async with client:
        data = await fetch_data_with_comments('@asbwf')  # ✅ Fetch data
        save_to_excel(data)  # ✅ Save as Excel (.xlsx)

asyncio.run(main())

In [ ]:
import pandas as pd

datasets = [
    "Youcef YD - العملات الرقميه 🏆🏅.xlsx",
    "أهداف المباريات _ YSM Sports.xlsx",
    "برمجة وتطوير.xlsx",
    "عصير الكتب - PDF.xlsx",
    "ناشيونال جيوغرافيگ.xlsx"
]

frames = [pd.read_excel(file) for file in datasets]

merged_df = pd.concat(frames, ignore_index=True, sort=False)

output_path = "merged_dataset.xlsx"
merged_df.to_excel(output_path, index=False)

print(f"Merged dataset saved to: {output_path}")

In [ ]:
import pandas as pd

file_path = 'modified_dataset.xlsx'
xls = pd.ExcelFile(file_path)

df = xls.parse(xls.sheet_names[0])

post_columns = [
    'Post ID', 'Post Content', 'Post Time', 'Views', 'Forwards', 'Fire (Post)',
    'Saluting Face (Post)', 'Pray (Post)', 'Trophy (Post)', 'Clapping (Post)',
    'Party (Post)', 'Face with Hearts (Post)', 'Thumbs Up (Post)', 'Heart (Post)',
    'Word Count', 'Average Words per Sentence'
]


comment_columns = [
  'Post ID','Comment ID', 'Comment Text', 'Comment Time', 'Heart (Comment)', 'Thumbs Up (Comment)'
]

posts_df = df[post_columns].drop_duplicates(subset=['Post ID'])
comments_df = df[comment_columns].dropna(subset=['Comment ID'])

posts_file = 'posts.xlsx'
comments_file = 'commentsID.xlsx'

comments_df.to_excel(comments_file, index=False)

print(f"Posts saved to: {posts_file}")
print(f"Comments saved to: {comments_file}")

In [ ]:
import pandas as pd
import re
import string

file_path = "posts.xlsx"
df = pd.read_excel(file_path)

text_column = "Post Content" if "Post Content" in df.columns else df.columns[0]

narrative_keywords = {
    "Engage": ["يا مغيث", "نداء", "الغضب", "التفاعل", "معركة"],
    "Explain": ["معاني", "سبب", "شرح", "تفاصيل", "أسباب", "توضيح", "معنى"],
    "Excite": ["حماس", "فرح", "مجد", "نصر", "افتخار"],
    "Enhance": ["دعم", "تشجيع", "تعزيز", "وعي", "توعية", "تحفيز"],
    "Dismiss": ["مبالغ", "تضخيم", "تافه", "ليس مهم", "وهم", "فقاعة", "تفاهة"],
    "Distort": ["تحريف", "تشويه", "تضليل", "كذب", "خداع"],
    "Dismay": ["قلق", "حزن", "رعب", "ألم", "دموع"],
    "Distract": ["غير ذي صلة", "تحويل الأنظار", "موضوع آخر", "بعيد عن", "لا يرتبط"]
}

def normalized_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", "", text)
        return text
    return ""

def classify_narrative(content):
    if pd.isna(content) or not isinstance(content, str) or content.strip() == "":
        return None
    content = normalized_text(content)
    for label, keywords in narrative_keywords.items():
        if any(keyword in content for keyword in keywords):
            return label
    return "Natural"

df["Narrative Manipulation"] = df[text_column].apply(classify_narrative)

output_path = "narrative_manipulation_posts.xlsx"
df.to_excel(output_path, index=False)

print(f"Narrative manipulation analysis complete. Results saved to {output_path}")

In [ ]:
import pandas as pd

file_path = "narrative_manipulation_posts.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

against_keywords = [
    "الغاشم", "لا يمكن", "عنيف", "العدوان",
    "تحذير", "خطر", "مشكلة", "انتقاد", "سلبي", "سقوط","سلبي","تقليل","تخفيض"
    "هجوم", "معارضة", "احتجاج", "تهديد", "تحريض", "معاداة", "خسارة"
]

favor_keywords = [
    "انتصار", "دعم", "نجاح", "إشادة", "موافقة", "احتفال", "تأييد","تزويد","سرعة",
    "مساندة", "حفل", "فرح", "نشر", "تمكين", "معقول", "فوز", "خدمة", "يخدم", "يدعم","ايجابي","إيجابي","وووو","يييي"
]

def label_post(row):
    content = str(row['Post Content'])
    content_lower = content.lower() if content != 'nan' else ""

    # down_broken_reactions = row.get('Thumbs Down (Post)', 0) + row.get('Broken Heart (Post)', 0)
    # heart_thumbs_reactions = row.get('Heart (Post)', 0) + row.get('Thumbs Up (Post)', 0)

    if any(keyword in content_lower for keyword in against_keywords):
            return "Against"
    elif any(keyword in content_lower for keyword in favor_keywords):
            return "Favor"

    return "Natural"

df['Label'] = df.apply(label_post, axis=1)

output_path = "labeled_narrative_manipulation_posts.xlsx"
df.to_excel(output_path, index=False)




In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

file_path = 'classified_eppm_posts.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

post_ids = df['Post ID']
post_contents = df['Post Content'].fillna('').str.lower()

tfidf_vectorizer = TfidfVectorizer(stop_words=None)

tfidf_matrix = tfidf_vectorizer.fit_transform(post_contents)

cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

similarity_threshold = 0.8

similar_posts = []
for i in range(cosine_sim_matrix.shape[0]):
    for j in range(i + 1, cosine_sim_matrix.shape[1]):
        if cosine_sim_matrix[i, j] >= similarity_threshold:
            similar_posts.append({
                'Post ID 1': post_ids[i],
                'Post ID 2': post_ids[j],
                'Similarity Score': cosine_sim_matrix[i, j],
                'Post1 Content': post_contents[i],
                'Post2 Content': post_contents[j]
            })

similar_posts_df = pd.DataFrame(similar_posts)

if not similar_posts_df.empty:
    output_file_path = 'similar_posts_results_full_content.xlsx'
    similar_posts_df.to_excel(output_file_path, index=False)
    print(f"Results saved to {output_file_path}")
else:
    print("No highly similar posts detected based on the chosen threshold.")

if similar_posts:
    similarity_scores = [entry['Similarity Score'] for entry in similar_posts]
    min_similarity = min(similarity_scores)
    max_similarity = max(similarity_scores)
    avg_similarity = sum(similarity_scores) / len(similarity_scores)
    print(f"Min: {min_similarity:.2f}, Max: {max_similarity:.2f}, Avg: {avg_similarity:.2f}")


In [ ]:
import pandas as pd

main_file_path = 'classified_eppm_posts.xlsx'
df_main = pd.read_excel(main_file_path, sheet_name='Sheet1')

similarity_file_path = 'similar_posts_results_full_content.xlsx'
df_similarity = pd.read_excel(similarity_file_path)

most_similar_dict = {}

for _, row in df_similarity.iterrows():
    post1, post2, sim_score = row['Post ID 1'], row['Post ID 2'], row['Similarity Score']

    if post1 not in most_similar_dict or sim_score > most_similar_dict[post1][1]:
        most_similar_dict[post1] = (post2, sim_score)

    if post2 not in most_similar_dict or sim_score > most_similar_dict[post2][1]:
        most_similar_dict[post2] = (post1, sim_score)

df_main['Most Similar Post ID'] = df_main['Post ID'].map(lambda x: most_similar_dict.get(x, (None, None))[0])
df_main['Highest Similarity Score'] = df_main['Post ID'].map(lambda x: most_similar_dict.get(x, (None, None))[1])

df_main['Most Similar Post ID'].fillna('No Match', inplace=True)
df_main['Highest Similarity Score'].fillna(0, inplace=True)

output_file = 'updated_post_narrative_with_similarity.xlsx'
df_main.to_excel(output_file, index=False)

print(f"Updated dataset saved to: {output_file}")


In [ ]:
import pandas as pd

file_path = 'labeled_narrative_manipulation_posts.xlsx'
df = pd.read_excel(file_path)

def elaboration_likelihood_model(content):
    if pd.isna(content):
        return "Other"

    if any(keyword in content for keyword in ["تصريحات", "التصريحات", "تصريح", "بيان", "أخبار", "التصريح", "معلومات", "الرسالة", "التفاصيل", "الحق", "يشرح", "كيف", "ما معنى"]):
        return "Information"
    elif any(keyword in content for keyword in ["?", "هل", "ماذا لو", "??", "؟؟", "؟", "!", "!!"]):
        return "Question"
    elif any(keyword in content for keyword in ["دعم", "تتبرع", "مظاهرة", "شارك", "بمشاركة", "تبرع", "والتبرع", "المشاركة", "دعما", "التبرع", "لتبرع", "مشاركته", "بتبرع", "التبرعات", "للمشاركة", "ودعما", "بالتبرع", "الدعم", "يدعم", "يشارك", "يساند", "اسناد", "إسناد", "دعماً", "مشاركة"]):
        return "Participation"
    elif any(keyword in content for keyword in ["زعيم", "لقائد", "سياسي", "قائد", "قائدنا", "القائد", "قادة", "القادة", "للقائد", "السياسي", "السياسية", "مشاهير", "الرئيس", "السنوار", "ابو عبيدة", "حزب الله", "القسام", "حماس", "نصرالله"]):
        return "Celebrity"
    elif any(keyword in content for keyword in ["نصرة", "نصرالله", "دعاء", "نصر", "إلهام", "النصر", "أمل", "اماني", "أماني", "المفاجأة"]):
        return "Inspiration"
    elif any(keyword in content for keyword in ["ضحك", "مزاح", "سخرية", "المضحك", "للسخرية", "بتمسخرو", "ههههههه", "ههههه", "ha", "هههههههههههه", "ضحكتني", "مسخرة", "ههه", "هههههههههههههههه"]):
        return "Humor/Sarcasm"
    elif any(keyword in content for keyword in ["ومشاهد", "شهادة", "الشهادة", "المشاهد", "رواية", "شاهد", "مشاهد", "قصة", "مشهد", "الحلم", "رواية"]):
        return "Anecdotal/Story"
    else:
        return "Information"

df['elaboration_likelihood'] = df['Post Content'].apply(elaboration_likelihood_model)

output_file = 'categorized_posts.xlsx'
df.to_excel(output_file, index=False)

print(f"Categorized posts saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset.xlsx'
df = pd.read_excel(file_path)

def social_judgment_theory(content):
    if pd.isna(content):
        return ""

    if any(keyword in content for keyword in ["دواء", "صحة", "مرض", "شفاء", "علاج","طب","تمريض","اصابة"]):
        return "Health/Evidence-based Health Information"
    elif any(keyword in content for keyword in ["النبي", "القرآن", "الإسلام", "دين", "صلاة", "الدين", "اليهودية",
                                                "الجنة", "اليهود", "مسلمين", "مسلمون", "إسلام","المسيحية"]):
        return "Religion"
    elif any(keyword in content for keyword in ["اختيار", "قرار", "حرية", "نظام", "حقوق", "رفاهية","امكانية","خيار"]):
        return "Choice"
    elif any(keyword in content for keyword in ["حكومة", "سياسي", "احتلال", "دولة", "جيش", "ثورة", "الدولة",
                                                "سياسة", "السياسة", "المعتقلين","ترامب","بايدن","دولي"]):
        return "Political"
    elif any(keyword in content for keyword in ["الله","تضحية", "إيثار", "كرم", "نصر", "دعم"]):
        return "Altruism"
    else:
        return ""

df['social_judgment'] = df['Post Content'].apply(social_judgment_theory)

output_file = 'semi_final_Stage4_dataset3.xlsx'
df.to_excel(output_file, index=False)

print(f"Categorized posts saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset3.xlsx'
df = pd.read_excel(file_path)

def classify_eppm(content):
    if pd.isna(content):
        return "Other"

    if any(keyword in content for keyword in [
        "خطر", "كارثة", "تهديد", "خسارة", "دمار", "مأساة", "موت", "وفيات",
        "قصف", "دماء", "حزن", "عذاب", "معاناة", "كارثة", "مجزرة", "خراب",
        "فقد", "جراح", "نار", "هلاك", "ظلم", "ألم", "ضياع"
    ]):
        return "Perceived Severity (EPPM-1)"

    elif any(keyword in content for keyword in [
        "قد يصيب", "معرض", "يصيب", "مرض", "أثر", "يصابون", "خوف", "قلق",
        "ضعف", "خطر داهم", "يتعرض", "تهديدات", "فقدان", "يهدد", "مهدد",
        "انهيار", "خطر محدق", "قريب", "ضحية","توتر"
    ]):
        return "Perceived Susceptibility (EPPM-2)"

    elif any(keyword in content for keyword in [
        "قادر", "يمكن", "يستطيع", "نستطيع", "حل", "امكانية", "القدرة", "التصدي",
        "نصمد", "ننتصر", "نحمي", "نقف", "ثبات", "عزم", "تصدي", "مواجهة",
        "تحقيق", "صمود", "إصرار", "نتقدم", "نجاح"
    ]):
        return "Perceived Self-Efficacy (EPPM-3)"

    elif any(keyword in content for keyword in [
        "فعالة", "فعالية", "نجاح", "نتيجة", "الاستجابة", "أفضل", "مناسب",
        "تحرير", "انتصار", "جدوى", "رد قوي", "تأثير", "حلول ناجحة", "نتائج إيجابية",
        "أمل", "فائدة", "تحسن", "استجابة فعالة", "نصر", "بذل الجهود", "ثمار","فوز"
    ]):
        return "Perceived Response-Efficacy (EPPM-4)"

    else:
        return ""

df['EPPM Category'] = df['Post Content'].apply(classify_eppm)

output_file = 'semi_final_Stage4_dataset4.xlsx'
df.to_excel(output_file, index=False)

print(f"Classified EPPM posts saved to: {output_file}")

In [ ]:
import pandas as pd
import re
import string

file_path = "comments.xlsx"
df = pd.read_excel(file_path)

text_column = "Comment Text" if "Comment Text" in df.columns else df.columns[0]

narrative_keywords = {
    "Engage": ["يا مغيث", "نداء", "الغضب", "التفاعل", "معركة","يا","يرتبط","ارتباط","مرتبط","يتعلق","تعليق"],
    "Explain": ["معاني", "سبب", "شرح", "تفاصيل", "أسباب", "توضيح", "معنى","هو"],
    "Excite": ["حماس", "فرح", "مجد", "نصر", "افتخار","فوز","ربح"],
    "Enhance": ["دعم", "تشجيع", "تعزيز", "وعي", "توعية", "تحفيز","زيادة","تزويد"],
    "Dismiss": ["مبالغ", "تضخيم", "تافه", "ليس مهم", "وهم", "فقاعة", "تفاهة","مستحيل"],
    "Distort": ["تحريف", "تشويه", "تضليل", "كذب", "خداع","مش ممكن"],
    "Dismay": ["قلق", "حزن", "رعب", "ألم", "دموع","الم","خسارة","فقد"],
    "Distract": ["غير ذي صلة", "تحويل الأنظار", "موضوع آخر", "بعيد عن", "لا يرتبط"]
}

def normalized_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", "", text)
        return text
    return ""

def classify_narrative(content):
    if pd.isna(content) or not isinstance(content, str) or content.strip() == "":
        return None
    content = normalized_text(content)
    for label, keywords in narrative_keywords.items():
        if any(keyword in content for keyword in keywords):
            return label
    return "Natural"

df["Narrative Manipulation (Comment)"] = df[text_column].apply(classify_narrative)

output_path = "/content/comment/narrative_manipulation_comment.xlsx"
df.to_excel(output_path, index=False)

print(f"Narrative manipulation analysis complete. Results saved to {output_path}")

In [ ]:
import pandas as pd

file_path = "/content/comment/narrative_manipulation_comment.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

against_keywords = [
    "الغاشم", "لا يمكن", "عنيف", "العدوان",
    "تحذير", "خطر", "مشكلة", "انتقاد", "سلبي", "سقوط","سلبي","تقليل","تخفيض"
    "هجوم", "معارضة", "احتجاج", "تهديد", "تحريض", "معاداة", "خسارة"
]

favor_keywords = [
    "انتصار", "دعم", "نجاح", "إشادة", "موافقة", "احتفال", "تأييد","تزويد","سرعة",
    "مساندة", "حفل", "فرح", "نشر", "تمكين", "معقول", "فوز", "خدمة", "يخدم", "يدعم","ايجابي","إيجابي","وووو","يييي"
]

def label_post(row):
    content = str(row['Comment Text'])
    content_lower = content.lower() if content != 'nan' else ""

    if any(keyword in content_lower for keyword in against_keywords):
            return "Against"
    elif any(keyword in content_lower for keyword in favor_keywords):
            return "Favor"

    return "Natural"

df['Stance (comment)'] = df.apply(label_post, axis=1)

output_path = "/content/comment/labeled_narrative_manipulation_posts.xlsx"
df.to_excel(output_path, index=False)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

file_path = "/content/comment/labeled_narrative_manipulation_posts.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

post_ids = df['Comment ID']
post_contents = df['Comment Text'].fillna('').str.lower()

tfidf_vectorizer = TfidfVectorizer(stop_words=None)

tfidf_matrix = tfidf_vectorizer.fit_transform(post_contents)

cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

similarity_threshold = 0.8

similar_posts = []
for i in range(cosine_sim_matrix.shape[0]):
    for j in range(i + 1, cosine_sim_matrix.shape[1]):
        if cosine_sim_matrix[i, j] >= similarity_threshold:
            similar_posts.append({
                'Comment ID 1': post_ids[i],
                'Comment ID 2': post_ids[j],
                'Similarity Score': cosine_sim_matrix[i, j],
                'Comment1 Text': post_contents[i],
                'Comment2 Text': post_contents[j]
            })

similar_posts_df = pd.DataFrame(similar_posts)

if not similar_posts_df.empty:
    output_file_path = '/content/comment/similar_posts_results_full_content.xlsx'
    similar_posts_df.to_excel(output_file_path, index=False)
    print(f"Results saved to {output_file_path}")
else:
    print("No highly similar posts detected based on the chosen threshold.")
if similar_posts:
    similarity_scores = [entry['Similarity Score'] for entry in similar_posts]
    min_similarity = min(similarity_scores)
    max_similarity = max(similarity_scores)
    avg_similarity = sum(similarity_scores) / len(similarity_scores)
    print(f"Min: {min_similarity:.2f}, Max: {max_similarity:.2f}, Avg: {avg_similarity:.2f}")

In [ ]:
import pandas as pd

main_file_path = "/content/comment/labeled_narrative_manipulation_posts.xlsx"
df_main = pd.read_excel(main_file_path, sheet_name='Sheet1')

similarity_file_path = '/content/comment/similar_posts_results_full_content.xlsx'
df_similarity = pd.read_excel(similarity_file_path)

most_similar_dict = {}

for _, row in df_similarity.iterrows():
    post1, post2, sim_score = row['Comment ID 1'], row['Comment ID 2'], row['Similarity Score']

    if post1 not in most_similar_dict or sim_score > most_similar_dict[post1][1]:
        most_similar_dict[post1] = (post2, sim_score)

    if post2 not in most_similar_dict or sim_score > most_similar_dict[post2][1]:
        most_similar_dict[post2] = (post1, sim_score)

df_main['Most Similar Comment ID'] = df_main['Comment ID'].map(lambda x: most_similar_dict.get(x, (None, None))[0])
df_main['Highest Similarity Score'] = df_main['Comment ID'].map(lambda x: most_similar_dict.get(x, (None, None))[1])

df_main['Most Similar Comment ID'].fillna('No Match', inplace=True)
df_main['Highest Similarity Score'].fillna(0, inplace=True)

output_file = '/content/comment/updated_post_narrative_with_similarity.xlsx'
df_main.to_excel(output_file, index=False)

print(f"Updated dataset saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = '/content/comment/updated_post_narrative_with_similarity.xlsx'
df = pd.read_excel(file_path)

def elaboration_likelihood_model(content):
    if pd.isna(content):
        return "Other"

    if any(keyword in content for keyword in ["تصريحات", "التصريحات", "تصريح", "بيان", "أخبار", "التصريح", "معلومات", "الرسالة", "التفاصيل", "الحق", "يشرح", "كيف", "ما معنى"]):
        return "Information"
    elif any(keyword in content for keyword in ["?", "هل", "ماذا لو", "??", "؟؟", "؟", "!", "!!"]):
        return "Question"
    elif any(keyword in content for keyword in ["دعم", "تتبرع", "مظاهرة", "شارك", "بمشاركة", "تبرع", "والتبرع", "المشاركة", "دعما", "التبرع", "لتبرع", "مشاركته", "بتبرع", "التبرعات", "للمشاركة", "ودعما", "بالتبرع", "الدعم", "يدعم", "يشارك", "يساند", "اسناد", "إسناد", "دعماً", "مشاركة"]):
        return "Participation"
    elif any(keyword in content for keyword in ["زعيم", "لقائد", "سياسي", "قائد", "قائدنا", "القائد", "قادة", "القادة", "للقائد", "السياسي", "السياسية", "مشاهير", "الرئيس", "السنوار", "ابو عبيدة", "حزب الله", "القسام", "حماس", "نصرالله"]):
        return "Celebrity"
    elif any(keyword in content for keyword in ["نصرة", "نصرالله", "دعاء", "نصر", "إلهام", "النصر", "أمل", "اماني", "أماني", "المفاجأة"]):
        return "Inspiration"
    elif any(keyword in content for keyword in ["ضحك", "مزاح", "سخرية", "المضحك", "للسخرية", "بتمسخرو", "ههههههه", "ههههه", "ha", "هههههههههههه", "ضحكتني", "مسخرة", "ههه", "هههههههههههههههه"]):
        return "Humor/Sarcasm"
    elif any(keyword in content for keyword in ["ومشاهد", "شهادة", "الشهادة", "المشاهد", "رواية", "شاهد", "مشاهد", "قصة", "مشهد", "الحلم", "رواية"]):
        return "Anecdotal/Story"
    else:
        return "Information"

df['elaboration_likelihood (Comment)'] = df['Comment Text'].apply(elaboration_likelihood_model)

output_file = '/content/comment/categorized_posts.xlsx'
df.to_excel(output_file, index=False)

print(f"Categorized posts saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset4.xlsx'
df = pd.read_excel(file_path)

def social_judgment_theory(content):
    if pd.isna(content):
        return ""

    if any(keyword in content for keyword in ["دواء", "صحة", "مرض", "شفاء", "علاج","طب","تمريض","اصابة"]):
        return "Health/Evidence-based Health Information"
    elif any(keyword in content for keyword in ["النبي", "القرآن", "الإسلام", "دين", "صلاة", "الدين", "اليهودية",
                                                "الجنة", "اليهود", "مسلمين", "مسلمون", "إسلام","المسيحية"]):
        return "Religion"
    elif any(keyword in content for keyword in ["اختيار", "قرار", "حرية", "نظام", "حقوق", "رفاهية","امكانية","خيار"]):
        return "Choice"
    elif any(keyword in content for keyword in ["حكومة", "سياسي", "احتلال", "دولة", "جيش", "ثورة", "الدولة",
                                                "سياسة", "السياسة", "المعتقلين","ترامب","بايدن","دولي"]):
        return "Political"
    elif any(keyword in content for keyword in ["الله","تضحية", "إيثار", "كرم", "نصر", "دعم"]):
        return "Altruism"
    else:
        return ""

df['social_judgment (comment)'] = df['Comment Text'].apply(social_judgment_theory)

output_file = 'semi_final_Stage4_dataset5.xlsx'
df.to_excel(output_file, index=False)

print(f"Categorized posts saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset5.xlsx'
df = pd.read_excel(file_path)

def classify_eppm(content):
    if pd.isna(content):
        return ""

    if any(keyword in content for keyword in [
        "خطر", "كارثة", "تهديد", "خسارة", "دمار", "مأساة", "موت", "وفيات",
        "قصف", "دماء", "حزن", "عذاب", "معاناة", "كارثة", "مجزرة", "خراب",
        "فقد", "جراح", "نار", "هلاك", "ظلم", "ألم", "ضياع"
    ]):
        return "Perceived Severity (EPPM-1)"

    elif any(keyword in content for keyword in [
        "قد يصيب", "معرض", "يصيب", "مرض", "أثر", "يصابون", "خوف", "قلق",
        "ضعف", "خطر داهم", "يتعرض", "تهديدات", "فقدان", "يهدد", "مهدد",
        "انهيار", "خطر محدق", "قريب", "ضحية","توتر"
    ]):
        return "Perceived Susceptibility (EPPM-2)"

    elif any(keyword in content for keyword in [
        "قادر", "يمكن", "يستطيع", "نستطيع", "حل", "امكانية", "القدرة", "التصدي",
        "نصمد", "ننتصر", "نحمي", "نقف", "ثبات", "عزم", "تصدي", "مواجهة",
        "تحقيق", "صمود", "إصرار", "نتقدم", "نجاح"
    ]):
        return "Perceived Self-Efficacy (EPPM-3)"

    elif any(keyword in content for keyword in [
        "فعالة", "فعالية", "نجاح", "نتيجة", "الاستجابة", "أفضل", "مناسب",
        "تحرير", "انتصار", "جدوى", "رد قوي", "تأثير", "حلول ناجحة", "نتائج إيجابية",
        "أمل", "فائدة", "تحسن", "استجابة فعالة", "نصر", "بذل الجهود", "ثمار","فوز"
    ]):
        return "Perceived Response-Efficacy (EPPM-4)"

    else:
        return ""

df['EPPM Category (Comment)'] = df['Comment Text'].apply(classify_eppm)

output_file = 'semi_final_Stage4_dataset6.xlsx'
df.to_excel(output_file, index=False)

print(f"Classified EPPM posts saved to: {output_file}")


In [ ]:
import pandas as pd

posts_file = 'posts_after_edits.xlsx'
comments_file = 'comments_after_edits.xlsx'

posts_df = pd.read_excel(posts_file)
comments_df = pd.read_excel(comments_file)

merged_df = posts_df.merge(comments_df, on='Post ID', how='left')

merged_file = 'semi_final_Stage4_dataset.xlsx'
merged_df.to_excel(merged_file, index=False)

print(f"Merged dataset saved to: {merged_file}")


In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset.xlsx'
df = pd.read_excel(file_path)

post_columns = ['Post ID', 'Post Content', 'Post Time', 'Views', 'Forwards', 'Fire (Post)',
                'Crying Face', 'Pray (Post)', 'Trophy (Post)', 'Clapping (Post)', 'Party (Post)',
                'With Heart', 'Thumbs Up', 'Heart (Post)', 'Word Count', 'Words per Manip',
                'Label', 'elaboration_likelihood', 'EPPM Category']

df.loc[df.duplicated(subset=['Post ID'], keep='first'), post_columns] = ''

output_file = 'cleaned_full_dataset.xlsx'
df.to_excel(output_file, index=False)

print(f"Cleaned dataset saved to: {output_file}")


In [ ]:
import pandas as pd

file_path = 'semi_final_Stage4_dataset.xlsx'
df = pd.read_excel(file_path)

post_columns = ['Post ID', 'Post Content', 'Post Time', 'Views', 'Forwards', 'Fire (Post)',
                'Crying Face', 'Pray (Post)', 'Trophy (Post)', 'Clapping (Post)', 'Party (Post)',
                'With Heart', 'Thumbs Up', 'Heart (Post)', 'Word Count', 'Words per Manip',
                'Label', 'ation_like_label', 'PM Categ']

comment_counts = df.groupby('Post Content').size().reset_index(name='Comment Count')

df = df.merge(comment_counts, on='Post Content', how='left')

df.loc[df.duplicated(subset=['Post Content'], keep='first'), post_columns] = ''

output_file = 'cleaned_dataset.xlsx'
df.to_excel(output_file, index=False)

print(f"Cleaned dataset saved to: {output_file}")

In [ ]:
import pandas as pd

file_path = 'new_full_dataset.xlsx'
df = pd.read_excel(file_path)

columns_to_extract = ['Post Content', 'Post ID']
df_extracted = df[columns_to_extract]

output_path = 'extracted_posts.xlsx'
df_extracted.to_excel(output_path, index=False)

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

file_path = "extracted_posts.xlsx"
df = pd.read_excel(file_path)

df = df[['Post Content']].dropna().reset_index(drop=True)

def clean_arabic_text(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ي', 'ى', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

df["Cleaned Post"] = df["Post Content"].apply(clean_arabic_text)

def label_post(text):
    central_keywords = ["دراسة", "بحث", "إحصائية", "بيانات", "إحصاء", "لأن", "بسبب", "إذا", "بحسب", "وفقًا ل"]

    peripheral_keywords = ["كارثي", "مخيف", "خطير", "مصيبة", "مرعب", "يقول الخبراء", "إما معنا أو ضدنا", "تهديد وجودي"]

    if any(word in text for word in central_keywords):
        return "Central Cue"
    elif any(word in text for word in peripheral_keywords):
        return "Peripheral Cue"
    else:
        return "Neutral"

df["Label"] = df["Cleaned Post"].apply(label_post)

df = df[df["Label"] != "Neutral"].reset_index(drop=True)

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X = vectorizer.fit_transform(df["Cleaned Post"])
y = df["Label"].map({"Central Cue": 0, "Peripheral Cue": 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

df["Predicted Label"] = model.predict(vectorizer.transform(df["Cleaned Post"]))
df["Predicted Label"] = df["Predicted Label"].map({0: "Central Cue", 1: "Peripheral Cue"})

df.to_excel("classified_posts.xlsx", index=False)
print("Classification Completed! Results saved to classified_posts.xlsx")